 # Instalando el ambiente

1. Create un nuevo virtualenv, activalo y actualizá `pip`: `python -m venv venv && source venv/bin/activate.fish && python -m pip install --upgrade pip setuptools`,
2. Instalá *Jupyter Lab*: `python -m pip install jupyterlab`,
3. Instalá las \"Google libs\": `python -m pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib`,
4. Creá un link simbólico que se llame `credentials.json` al JSON que tiene tus credenciales bajadas,

Se crea con el objetivo de guardar todas las librerias y dependencias necesarias que requiera el proyecto. 

# Etapa 1 Extración de los datos de los pdf
En este Jupyter notebook creo los bloques con todos los ejemplos para recopilar los direrentes tipos de datos en los pdf.
A continuación menciono que no mer sirvio y porque lo descarte. 


### 1) No me sirvio utilizar PyPDF2
Es complicado de entender la información ya que me trae metadatos como la fecha de creación del archivo, autor y demás, por lo anterior concluí no me sirve para mi desarrollo. 
Intenté con otro código pero me sigue generando un error ya teniendo instalado el paquete de PdfReader

In [2]:
import PyPDF2 
import PdfReader
pdf_file_obj= open("D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\ARG\\PruebaAR02_IVA_DDJJ.pdf", "rb")
# pdf_read= PyPDF2.PdfFileReader(pdf_file_obj)
# info =pdf_read.getDocumentInfo()
# print(info)
# print(pdf_read.pages)
# print(pdf_read.pages[0].extract_text())

reader = PdfReader(pdf_file_obj)
text = reader.pages[0].extract_text()
assert "Total" in text, text

ModuleNotFoundError: No module named 'PyPDF2'

In [ ]:
from PyPDF2 import PdfFileReader, PdfFileWriter,PdfFileMerger
pdf_file_obj= open("D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\ARG\\PruebaAR02_IVA_DDJJ.pdf", "rb")
template = PdfFileReader(pdf_file_obj)
output = PdfFileWriter()

for i in range(template.getNumPages()):
   page = template.getPage(i)
   print(page)
   page.mergePage(watermark.getPage(0))
   output.addPage(page)
# file = open('waterMarked_PDF.pdf', 'wb')
# output.write(file)

: 

### 2) No me sirvio utilizar fitz
Intento con html pero no tiene un buen formato para extraer información. 

In [ ]:
import fitz 
doct=fitz.open("D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\COL\\VIMEO_SD_PRESENTADA_1BIM_2022.pdf")
pag=doct.load_page(0)
resultado=pag.search_for("Transactions ")
extraer = pag.get_textbox(resultado)
pag.get_text("xhtml")
for line in convertido:
    # print(line)
# print(convertido)
doct_html=open("documento.html","w")
doct_html.write(convertido)
doct_html.close()
print(doct.get_toc())

: 

### 3) Que necesito y utilizo?

 * Leer los formularios para ello debo instalar "tabula" y "tabulate" para leer las tablas de los pdfs ya que los datos estan separados por tablas.
 * También necesito instalar Pandas para analizar los datos y adicional lo requiere Tabula.
 * Numpy sirve para analizar números.(aún no se ha utilizado) 

#### 3.1 Lectura de formatos

In [ ]:
from tabula import read_pdf
from tabulate import tabulate
import pandas as pd
import numpy as np
import re #  Es para la notación de cadena raw de Python para los patrones de Unicode (str)
import locale # separador de mil en US O UK

#**Formulario IvA-ARG
ruta_arg = "D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\ARG\\PruebaAR02_IVA_DDJJ.pdf"
ruta_col = "D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\COL\\Decla. IVA II BIM 2022 - CO10.PDF"
f_Arg = read_pdf(ruta_arg,multiple_tables=True)
f_Col_300 = read_pdf(ruta_col,multiple_tables=True)

##### 3.1.1 Lectura de todo el pdf Argentina, extracción y captura de cada item. 

In [ ]:
año_p= f_Arg[0].iloc[[1],[3]] # pendiente separar cuit y periodo fiscal 
año_p,len(año_p),type(año_p),año_p.values[0][0] # Identifico como recorrer la lineda y se accede por medio de una array
año_pf=año_p.values[0][0]
n=año_pf.split(" ")[0]
period= str(n.split("-")[0]) #ok periodo
año= int(n.split("-")[1]) #ok año


form_unido=f_Arg[0].iloc[[4],[0]] #pendiente separar formulario y razon social 
a= form_unido.values[0][0]
n_for = a.split(" ")[0] # ok formulario

razon_soc2= a.split(" ")[7]
razon_soc1= a.split(" ")[6]+" "+razon_soc2 # ok razon social


verifi= f_Arg[0].iloc[[3],[3]] #pendiente separar form, CUIT, actividad principal y N° verificador
cod_verif= verifi.values[0][0].split(" ")[1] # lo identifico, separo y accedo

#Lectura accede a la primera y segunda sección 
for fila in f_Arg[0].loc[11:].itertuples():#salo las primeras 11 filas
    print(fila[1], fila[4])



##### a) Como extraer CUIT de los pdf? (Argentina)
Debo importar "re" para buscar expresiones regulares

In [ ]:
reg = re.compile(r'[0-9]{2}-[0-9]{8}-[0-9]{1}') # Creo el expresion regular para encontrar el CUIT
id_RazonSocial = reg.search(str(f_Arg[0].iloc[[3],[3]])) # Acá eligo la celda donde se encuentra el CUIT junto con otro texto
print(id_RazonSocial.group(0)) # Imprimo el resultado CUIT

##### b) Como convertir un str a float? 
Debo de importar libreria "locale"

In [ ]:
#Saldo técnico a favor de AFIP
saldo_AFIP = f_Arg[0].iloc[[43],[3]] #Saldo de impuesto a favor de AFIP
sal_entid = saldo_AFIP.values[0][0].replace("." , "") #Accedo al valor pero tengo que limpiar datos para volverlo float 
#locale.setlocale(locale.LC_ALL, 'en_US.UTF-8') solo para . y ,  pero en entero
locale.setlocale(locale.LC_ALL, 'nl_NL') #valor str convertido a float 
valuo_gob= locale.atof(sal_entid)
print(type(valuo_gob), valuo_gob ) # Pendiente convertirlo en numero

#--------------------------------------
#Saldo técnico a favor del responsable
saldo_respon = f_Arg[0].iloc[[42],[3]] #Saldo de impuesto a favor de AFIP
sal_respon = saldo_respon.values[0][0]  #Accedo al valor pero tengo que limpiar datos para volverlo float 
locale.setlocale(locale.LC_ALL, 'nl_NL') #valor str convertido a float 
valuo_respon= locale.atof(sal_respon)
print(type(valuo_respon), valuo_respon ) # Pendiente convertirlo en numero


##### c) Fomulario de Colombia con dos divisiones 
Debo de importar libreria de pandas 
from seaborn "import load_dataset"

In [ ]:
#Lectura accede a la primera y segunda sección 
for fila in f_Col_300[0].itertuples():#salo las primeras 11 filas
    print(fila)
#print(f_Col_300)



In [ ]:
razon_soc1 = f_Col_300[0].iloc[9,[0]].values # linea 9 col 1

period= f_Col_300[0].iloc[3,[5]].values[0]
reg_per = re.compile(r'Período [1-6]{1}')
perio_f= reg_per.search(period)
valor = perio_f.group(0).split(' ')
valor_periodo = valor[1] # valor periodo asegurado
cod_verif = f_Col_300[0].iloc[4,[7]].values 
año =  f_Col_300[0].iloc[3,[0,1,2,3]].values  # linea verlo de otra forma sacar el año tiene str y  int
saldo_pagar= f_Col_300[0].iloc[52,[7]]
saldo_favor= f_Col_300[0].iloc[54,[8]]

print(valor_periodo, razon_soc1, año, saldo_pagar)
print(saldo_favor)




### 4) Libreria pdf pluber ##


En el proceso de revisión y ejecución de extracción de los datos elegí extraerlos con la libreria pdf plumber, es mucho mas legible el proceso de los datos.

Docuemntación
[https://github.com/jsvine/pdfplumber#extracting-form-values]


In [ ]:
import pdfplumber
import re
ruta_col = "D:\\Jazmin\\Archivos Personales\\Facultad\\Facultad Analista de Sistemas\\5 CUATRI\\4Practica Profesional\\Recibos\\COL\\Decla. IVA II BIM 2022 - CO10.PDF"

with pdfplumber.open(ruta_col) as pdf:
    pag = pdf.pages[0]
    text = pag.extract_text()
form= text.split('\n')[0]
form_n= text.split('\n')[3]
nombre = text.split('\n')[4]

   

# print( form, form_n)
# print(text)
#Renglon  y valor
# for m in re.finditer(r'[0-9]{2}\s(\d{1,3}[,]?)+', text):
#     print(m.group(0))

# for m in re.finditer(r'[0-9]{2}\s(\d{1,3}[,]?)+', text):
#     print(m.group(0))



# Lectura de formulario descripción, renglon y valor
for m in re.finditer(r'.+?\s\d{2}\s(\d{1,3}[,]?){1,}', text):
    print(m.group(0))

In [ ]:

# Obtener y limpiar datos e una linea
#a = re.compile(r'1. Año\s([0-9]\s){4}') 
#b = a.match(text)
b= re.search(r'Año\s([0-9]\s){4}', text)
año = re.search(r'([0-9]\s){4}',b.group(0)) 
periodo = re.search(r'([1-6]\s){1}',b.group(0)) 
print(año.group(0).replace(" ", ""), periodo.group(0).replace(" ", ""))



In [ ]:
from decimal import *

Context(2,2,)

print(Decimal("666,868,000", context={ }))

En el formato de de Mexico no leia correctamente el nombre y unia todas las palabras, se tuvo que revisar la libreria de pdfplumber


#.extract_text(x_tolerance=3, y_tolerance=3, layout=False, x_density=7.25, y_density=13, **kwargs)

Cuando layout=False: Agrega espacios donde la diferencia entre el x1 de un carácter y el x0 del siguiente es mayor que [x_tolerance]. Agrega caracteres de nueva línea donde la diferencia entre el doctop de un carácter y el doctop del siguiente es mayor que y_tolerance.

Referencia:

https://github.com/jsvine/pdfplumber#python-library


In [ ]:
from posixpath import split
import pdfplumber
import re
#  Es para la notación de cadena raw de Python para los patrones de Unicode (str)
import locale # separador de mil en US O UK


ruta_mex = "./../../../app/dataextraction/Recibos/MEX/MEX_VAT_Feb2022_Detail.pdf"

cconcepto= []
with pdfplumber.open(ruta_mex) as pdf:
    pag = pdf.pages[0]
    
    text = pag.extract_text()
    lineas =text.split('\n')

concepto ={}
print(text)
for linea in lineas:
    concepto.append(linea)

# Etapa 2 Creación Entorno Docker (ver together love)

Se crea configuraciones y ambiente para que se ejecute Docker se incorpora Django para que funcione aplicación. 
El proceso que se tuvo que realizar es el siguiente:

sin embargo se tuvo que incluir propias especificas de python.
1. Se agrega un [Dockerfile](https://docs.docker.com/engine/reference/builder/) para ejecutar el
aplicación con [Docker](https://docs.docker.com/get-started/).

2. Antes de ejecutar su aplicación con un contenedor acoplable, debe crear su imagen acoplable.
3. Descargue una imagen oficial de python y ingreso a  Docker 
4. La imagen no tiene soporte de requerimientos por eso incluyo adicional unas librerias propias de lo que voy a utilizar [Django] (https://docs.djangoproject.com/en/4.1/).Es por eso que hay un archivo `requirements.txt`.
5. El archivo `requirements.txt` se exportó con `poetry`.

## a) Que configiraciones adicionales tuve que realizar para mi base de datos?

1. Tuve que crear un Mock para simular la realidad, se incluyen unos parametros para que pueda leer los archivos. [Test_commands.py]

2. Se llama la funcion y la pasa [database = defaul] me crear una base al iniciar, y verfica que se llamo con  [assert_called.with]

3. Se crear archivo [wait_for_db.py] comando que verifica si esta **conectada** se crea una bandera [banderas_db.up]
Mientras que sea falso, salta la sentencia True generando directamente un **except** . Esta prueba me identifica que esta funcionando con un loop, sino conecta salta el error hasta que pueda conectar con la base de datos. 

4. Me genero un *error* "For production one .. paycocp2 advisc you distribution". Lo cual no me permitia visualizar si estaba corriendo bien la aplicacion. Probe varias cosas lo cual dejo como enseñanza:

    4.1 El comando **delta** se ingresa a la terminal para visualizar los requirimientos y agregarlos en el archivo. Comando [pp intall-r requerimientos.txt]

    4.2 Se tubo que ingresar libreias y cada vez que se incluia una debia actualizar la imagen [docker compose build].

    4.3 Cuando hace falta una configiracion en la base de datos se salta el error en el test porque no lo reconoce, me fallaba varias veces por que fataba por incluir el tample o los datos fuente de conexión DATABASE.  
    
    4.4 Importante, se deben agregar todas las carpetas para que los pueda encontrar "user" y "core" que son las aplicaciones propias que se agregan para que funcione la aplicaccion. En mi caso tuve que incluir  en el archivo requisitos se debe ingresar en la configuración mi APP Setting InstalV_APP. 

5. Se incluye los test donde comprueba errores mientras que se carga la aplicación, si da exito ejecuta el proceso de la APP si es FALSO, muestra como se deberia tratar tal error. 
    
    5.1 Creación de pruebas unitarias para los modulos de un [usuario] en [django]. El usuario registrarse con su emial y contraseña, el sistema internamente lo crea con **ID**.

    5.2 Prueba united donde se normaliza el email.

    5.3 Cuando se crea un usuario sin email retorna error. 


    

# Etapa 3 Generación de Indicadores y construcción dashboard  (visualizaciones)


## 3.1 Indicadores claves de desempeño (KPIS)


Variables que se tomaran como factores y unidades de medida para generar una estrategia de marketing.

Sirve para medir el rendimiento en las áreas, se analizan y evaluan los logros parciales y el objetivo final.

### 3.1.1.Diagnostico 
Comparar la situación real con la planificada para detectar desviaciones. 

#### a) Indicador Atraso en el cronograma
Para el proyecto esta medición se realizará con el <Atraso en el cronograma> se evaluará que tan eficiente fue en cada etapa del proceso del impuesto, estos son:

    * Fecha de vencimiento vs Fecha Revisión
    * Fecha de vencimiento vs Fecha Envio Cliente
    * Fecha de vencimiento vs Fecha Procesamiento (presentación / carga documento Ekchy)


#### b). Porcentaje de Ejecución
Esta métrica compara el alcance que lleva ejecutado (impuestos presentados y finalizados), con el que se debe llevar, según el plan de vencimiento de impuesto por pais y según periodo de vencimiento (Solo se realiza la comparación del periodo).

    * Total impuestos por hacer vs hechos (en el periodo)
    
### 3.1.2. Indicadores del tipo retrospectivos

Su medición aporta visibilidad respecto a lo que se ha hecho hasta el momento.Es decir, mira hacia atrás para recoger información sobre los tiempos que se han generado por periodos de rango del año según histórico. Para efectos del proyecto se realiza trimestralmente desde la fecha Enero a Marzo 2022. 

#### a) Cantidad de Entregables 

Esta métrica nos permite medir el avance del alcance del proyecto. 
Para efectos del proyecto se identifica los siguientes estados:

    * "OK Procesado": Es cuando finaliza todas las etapas el impuesto (presentado y pagado), es decir se realizó la Entrega. 
    * "Pendiente": Cuando está en espera para que culmine una etapa antes del final, por lo anterior no a finalizado totalmente. 
    * "No procesado": Cuando por alguna razón no finalizó el proceso  y cerro el periodo sin presentarse el impuesto.


    

In [1]:
"""Se realiza pruebas de tablas con json para simular base de datos de consulta"""

#https://csvjson.com/csv2json


# Extraccion, una lista de restros
lista_extraccExcel = [
  {
    "pais": "ARG",
    "id_razonsocial": "30-70892838-7",
    "nombre_empresa": "BSF SA",
    "periodo_fiscal": 2,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "11/3/2022",
    "Fecha_envio_cliente": "8/3/2022",
    "Fecha_revisado": "7/3/2022",
    "Revisor": "Juan Espinosa"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70877589-0",
    "nombre_empresa": "IAFH GLOBAL SA",
    "periodo_fiscal": 2,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/3/2022",
    "Fecha_envio_cliente": "12/3/2022",
    "Fecha_revisado": "11/3/2022",
    "Revisor": "Camilo Sarmiento"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013543200,
    "nombre_empresa": "AIRLAS",
    "periodo_fiscal": 2,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "19/3/2022",
    "Fecha_envio_cliente": "16/3/2022",
    "Fecha_revisado": "15/3/2022",
    "Revisor": "Juan Espinosa"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013161451,
    "nombre_empresa": "BELGRADNO",
    "periodo_fiscal": 2,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "12/3/2022",
    "Fecha_envio_cliente": "9/3/2022",
    "Fecha_revisado": "8/3/2022",
    "Revisor": "Camilo Sarmiento"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9014115458,
    "nombre_empresa": "POSTOBON",
    "periodo_fiscal": 2,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/3/2022",
    "Fecha_envio_cliente": "12/3/2022",
    "Fecha_revisado": "11/3/2022",
    "Revisor": "Flor Cardenas"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-69556267-1",
    "nombre_empresa": "DECISION SUPPORT SA",
    "periodo_fiscal": 2,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "9/3/2022",
    "Fecha_envio_cliente": "6/3/2022",
    "Fecha_revisado": "5/3/2022",
    "Revisor": "Flor Cardenas"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71245283-4",
    "nombre_empresa": "DYNAFLOWS SA",
    "periodo_fiscal": 2,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/3/2022",
    "Fecha_envio_cliente": "14/3/2022",
    "Fecha_revisado": "13/3/2022",
    "Revisor": "Maria Rivera"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70840691-7",
    "nombre_empresa": "SISTEMAS GLOBALES SA",
    "periodo_fiscal": 2,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "16/3/2022",
    "Fecha_envio_cliente": "13/3/2022",
    "Fecha_revisado": "12/3/2022",
    "Revisor": "Juan Espinosa"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9002185787,
    "nombre_empresa": "SISTEMAS COLOMBIA S.A.S",
    "periodo_fiscal": 2,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/3/2022",
    "Fecha_envio_cliente": "13/3/2022",
    "Fecha_revisado": "12/3/2022",
    "Revisor": "Camilo Sarmiento"
  },
  {
    "pais": "MEX",
    "id_razonsocial": "NPI620328DP8",
    "nombre_empresa": "NCS PEARSON, INC.",
    "periodo_fiscal": 2,
    "nombreFormulario": "IVA DIGITALES",
    "Fecha_vencimiento": "19/3/2022",
    "Fecha_envio_cliente": "17/3/2022",
    "Fecha_revisado": "16/3/2022",
    "Revisor": "Flor Cardenas"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71509409-2",
    "nombre_empresa": "AMDIPHARM LTD",
    "periodo_fiscal": 2,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "17/3/2022",
    "Fecha_envio_cliente": "14/3/2022",
    "Fecha_revisado": "13/3/2022",
    "Revisor": "Maria Rivera"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013543905,
    "nombre_empresa": "AIRBNB IRELAND UNLIMITED COMPANY",
    "periodo_fiscal": 2,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "19/3/2022",
    "Fecha_envio_cliente": "16/3/2022",
    "Fecha_revisado": "15/3/2022",
    "Revisor": "Juan Espinosa"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013161970,
    "nombre_empresa": "VIMEO.COM, INC.",
    "periodo_fiscal": 2,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "12/3/2022",
    "Fecha_envio_cliente": "9/3/2022",
    "Fecha_revisado": "8/3/2022",
    "Revisor": "Camilo Sarmiento"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9014115336,
    "nombre_empresa": "GLOBANT COLOMBIA S.A.S",
    "periodo_fiscal": 2,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/3/2022",
    "Fecha_envio_cliente": "12/3/2022",
    "Fecha_revisado": "11/3/2022",
    "Revisor": "Flor Cardenas"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70892838-7",
    "nombre_empresa": "BSF SA",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "11/2/2022",
    "Fecha_envio_cliente": "8/2/2022",
    "Fecha_revisado": "7/2/2022",
    "Revisor": "Juan Espinosa"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70877589-0",
    "nombre_empresa": "IAFH GLOBAL SA",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/2/2022",
    "Fecha_envio_cliente": "12/2/2022",
    "Fecha_revisado": "11/2/2022",
    "Revisor": "Camilo Sarmiento"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-69556267-1",
    "nombre_empresa": "DECISION SUPPORT SA",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "9/2/2022",
    "Fecha_envio_cliente": "6/2/2022",
    "Fecha_revisado": "5/2/2022",
    "Revisor": "Flor Cardenas"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71245283-4",
    "nombre_empresa": "DYNAFLOWS SA",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/2/2022",
    "Fecha_envio_cliente": "14/2/2022",
    "Fecha_revisado": "13/2/2022",
    "Revisor": "Maria Rivera"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70892838-7",
    "nombre_empresa": "BSF SA",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "11/2/2022",
    "Fecha_envio_cliente": "8/2/2022",
    "Fecha_revisado": "7/2/2022",
    "Revisor": "Juan Espinosa"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70877589-0",
    "nombre_empresa": "IAFH GLOBAL SA",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/2/2022",
    "Fecha_envio_cliente": "12/2/2022",
    "Fecha_revisado": "11/2/2022",
    "Revisor": "Camilo Sarmiento"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-69556267-1",
    "nombre_empresa": "DECISION SUPPORT SA",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "9/2/2022",
    "Fecha_envio_cliente": "6/2/2022",
    "Fecha_revisado": "5/2/2022",
    "Revisor": "Flor Cardenas"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71245283-4",
    "nombre_empresa": "DYNAFLOWS SA",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/2/2022",
    "Fecha_envio_cliente": "14/2/2022",
    "Fecha_revisado": "13/2/2022",
    "Revisor": "Maria Rivera"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70892838-7",
    "nombre_empresa": "BSF SA",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "11/2/2022",
    "Fecha_envio_cliente": "8/2/2022",
    "Fecha_revisado": "7/2/2022",
    "Revisor": "Juan Espinosa"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70877589-0",
    "nombre_empresa": "IAFH GLOBAL SA",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/2/2022",
    "Fecha_envio_cliente": "12/2/2022",
    "Fecha_revisado": "11/2/2022",
    "Revisor": "Camilo Sarmiento"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-69556267-1",
    "nombre_empresa": "DECISION SUPPORT SA",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "9/2/2022",
    "Fecha_envio_cliente": "6/2/2022",
    "Fecha_revisado": "5/2/2022",
    "Revisor": "Flor Cardenas"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71245283-4",
    "nombre_empresa": "DYNAFLOWS SA",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/2/2022",
    "Fecha_envio_cliente": "14/2/2022",
    "Fecha_revisado": "13/2/2022",
    "Revisor": "Maria Rivera"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70840691-7",
    "nombre_empresa": "SISTEMAS GLOBALES SA",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "16/2/2022",
    "Fecha_envio_cliente": "13/2/2022",
    "Fecha_revisado": "12/2/2022",
    "Revisor": "Juan Espinosa"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9002185787,
    "nombre_empresa": "SISTEMAS COLOMBIA S.A.S",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/2/2022",
    "Fecha_envio_cliente": "13/2/2022",
    "Fecha_revisado": "12/2/2022",
    "Revisor": "Camilo Sarmiento"
  },
  {
    "pais": "MEX",
    "id_razonsocial": "NPI620328DP8",
    "nombre_empresa": "NCS PEARSON, INC.",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA DIGITALES",
    "Fecha_vencimiento": "19/2/2022",
    "Fecha_envio_cliente": "17/2/2022",
    "Fecha_revisado": "16/2/2022",
    "Revisor": "Flor Cardenas"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71509409-2",
    "nombre_empresa": "AMDIPHARM LTD",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "17/2/2022",
    "Fecha_envio_cliente": "14/2/2022",
    "Fecha_revisado": "13/2/2022",
    "Revisor": "Maria Rivera"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013543905,
    "nombre_empresa": "AIRBNB IRELAND UNLIMITED COMPANY",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "19/2/2022",
    "Fecha_envio_cliente": "16/2/2022",
    "Fecha_revisado": "15/2/2022",
    "Revisor": "Juan Espinosa"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013161970,
    "nombre_empresa": "VIMEO.COM, INC.",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "12/2/2022",
    "Fecha_envio_cliente": "9/2/2022",
    "Fecha_revisado": "8/2/2022",
    "Revisor": "Camilo Sarmiento"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9014115336,
    "nombre_empresa": "GLOBANT COLOMBIA S.A.S",
    "periodo_fiscal": 1,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/2/2022",
    "Fecha_envio_cliente": "12/2/2022",
    "Fecha_revisado": "11/2/2022",
    "Revisor": "Flor Cardenas"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70892838-7",
    "nombre_empresa": "BSF SA",
    "periodo_fiscal": 12,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "11/1/2022",
    "Fecha_envio_cliente": "8/1/2022",
    "Fecha_revisado": "7/2/2022",
    "Revisor": "Juan Espinosa"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70877589-0",
    "nombre_empresa": "IAFH GLOBAL SA",
    "periodo_fiscal": 12,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/1/2022",
    "Fecha_envio_cliente": "12/1/2022",
    "Fecha_revisado": "11/2/2022",
    "Revisor": "Camilo Sarmiento"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-69556267-1",
    "nombre_empresa": "DECISION SUPPORT SA",
    "periodo_fiscal": 12,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "9/1/2022",
    "Fecha_envio_cliente": "6/1/2022",
    "Fecha_revisado": "5/2/2022",
    "Revisor": "Flor Cardenas"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71245283-4",
    "nombre_empresa": "DYNAFLOWS SA",
    "periodo_fiscal": 12,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/1/2022",
    "Fecha_envio_cliente": "14/1/2022",
    "Fecha_revisado": "13/2/2022",
    "Revisor": "Maria Rivera"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70840691-7",
    "nombre_empresa": "SISTEMAS GLOBALES SA",
    "periodo_fiscal": 12,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "16/1/2022",
    "Fecha_envio_cliente": "13/1/2022",
    "Fecha_revisado": "12/2/2022",
    "Revisor": "Juan Espinosa"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9002185787,
    "nombre_empresa": "SISTEMAS COLOMBIA S.A.S",
    "periodo_fiscal": 12,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/1/2022",
    "Fecha_envio_cliente": "13/1/2022",
    "Fecha_revisado": "12/2/2022",
    "Revisor": "Camilo Sarmiento"
  },
  {
    "pais": "MEX",
    "id_razonsocial": "NPI620328DP8",
    "nombre_empresa": "NCS PEARSON, INC.",
    "periodo_fiscal": 12,
    "nombreFormulario": "IVA DIGITALES",
    "Fecha_vencimiento": "19/1/2022",
    "Fecha_envio_cliente": "17/1/2022",
    "Fecha_revisado": "16/2/2022",
    "Revisor": "Flor Cardenas"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71509409-2",
    "nombre_empresa": "AMDIPHARM LTD",
    "periodo_fiscal": 12,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "17/1/2022",
    "Fecha_envio_cliente": "14/1/2022",
    "Fecha_revisado": "13/2/2022",
    "Revisor": "Maria Rivera"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013543905,
    "nombre_empresa": "AIRBNB IRELAND UNLIMITED COMPANY",
    "periodo_fiscal": 12,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "19/1/2022",
    "Fecha_envio_cliente": "16/1/2022",
    "Fecha_revisado": "15/2/2022",
    "Revisor": "Juan Espinosa"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013161970,
    "nombre_empresa": "VIMEO.COM, INC.",
    "periodo_fiscal": 12,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "12/1/2022",
    "Fecha_envio_cliente": "9/1/2022",
    "Fecha_revisado": "8/2/2022",
    "Revisor": "Camilo Sarmiento"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9014115336,
    "nombre_empresa": "GLOBANT COLOMBIA S.A.S",
    "periodo_fiscal": 12,
    "nombreFormulario": "IVA",
    "Fecha_vencimiento": "15/1/2022",
    "Fecha_envio_cliente": "12/1/2022",
    "Fecha_revisado": "11/2/2022",
    "Revisor": "Flor Cardenas"
  }
]

# vencimientos, una lista de restros
lista_procesadopdf= [
  {
    "pais": "ARG",
    "id_razonsocial": "30-70892838-7",
    "a–o": 2022,
    "periodo_fiscal": 2,
    "n_verificacion": "146752/201103",
    "saldoPagado": "3.313.414,59",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "12/9/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70877589-0",
    "a–o": 2022,
    "periodo_fiscal": 2,
    "n_verificacion": "146753/201104",
    "saldoPagado": "0,00",
    "saldoFavor": "200.000,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "12/9/2022",
    "Procesado": "Pendiente"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013543200,
    "a–o": 2022,
    "periodo_fiscal": 2,
    "n_verificacion": 3004648676658,
    "saldoPagado": "256.200.000,00",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "12/9/2022",
    "Procesado": "Pendiente"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013161451,
    "a–o": 2022,
    "periodo_fiscal": 2,
    "n_verificacion": 3004648676659,
    "saldoPagado": "102.124.000,00",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "12/9/2022",
    "Procesado": "Pendiente"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9014115458,
    "a–o": 2022,
    "periodo_fiscal": 2,
    "n_verificacion": 3004648676660,
    "saldoPagado": "4.520.000,00",
    "saldoFavor": "",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "12/9/2022",
    "Procesado": "Pendiente"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-69556267-1",
    "a–o": 2022,
    "periodo_fiscal": 2,
    "n_verificacion": "146754/201105",
    "saldoPagado": "5.635.000,00",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "12/9/2022",
    "Procesado": "Pendiente"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71245283-4",
    "a–o": 2022,
    "periodo_fiscal": 2,
    "n_verificacion": "146755/201106",
    "saldoPagado": "0,00",
    "saldoFavor": "300.000,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "12/9/2022",
    "Procesado": "Pendiente"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70840691-7",
    "a–o": 2022,
    "periodo_fiscal": 2,
    "n_verificacion": "146756/201107",
    "saldoPagado": "5.313.414,59",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "12/9/2022",
    "Procesado": "Pendiente"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9002185787,
    "a–o": 2022,
    "periodo_fiscal": 2,
    "n_verificacion": 3004648676658,
    "saldoPagado": "666.868.000,00",
    "saldoFavor": 0,
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "12/9/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "MEX",
    "id_razonsocial": "NPI620328DP8",
    "a–o": 2022,
    "periodo_fiscal": 2,
    "n_verificacion": 220060080649,
    "saldoPagado": "213,352",
    "saldoFavor": 0,
    "Nombre Impuesto": "IVA DIGITALES",
    "fecha_procesado": "12/9/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71509409-2",
    "a–o": 2022,
    "periodo_fiscal": 2,
    "n_verificacion": "146759/201110",
    "saldoPagado": "4.313.414,59",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "12/9/2022",
    "Procesado": "Pendiente"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013543905,
    "a–o": 2022,
    "periodo_fiscal": 2,
    "n_verificacion": 3004648676658,
    "saldoPagado": "256.200.000,00",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "12/9/2022",
    "Procesado": "Pendiente"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013161970,
    "a–o": 2022,
    "periodo_fiscal": 2,
    "n_verificacion": 3004648676659,
    "saldoPagado": "3.002.124.000,00",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "12/9/2022",
    "Procesado": "Pendiente"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9014115336,
    "a–o": 2022,
    "periodo_fiscal": 2,
    "n_verificacion": 3004648676660,
    "saldoPagado": "4.520.000,00",
    "saldoFavor": "",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "12/9/2022",
    "Procesado": "Pendiente"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70892838-7",
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": 3004648676661,
    "saldoPagado": "3.313.414,59",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "10/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70877589-0",
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": 3004648676662,
    "saldoPagado": "0,00",
    "saldoFavor": "200.000,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "14/2/2022",
    "Procesado": "Correcci—n"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-69556267-1",
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": 3004648676663,
    "saldoPagado": "5.635.000,00",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "8/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71245283-4",
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": 3004648676664,
    "saldoPagado": "0,00",
    "saldoFavor": "300.000,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "14/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70892838-7",
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": 3004648676665,
    "saldoPagado": "3.313.414,59",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "10/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70877589-0",
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": 3004648676666,
    "saldoPagado": "0,00",
    "saldoFavor": "200.000,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "14/2/2022",
    "Procesado": "Correcci—n"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-69556267-1",
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": 3004648676667,
    "saldoPagado": "5.635.000,00",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "8/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71245283-4",
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": "n_verificacion",
    "saldoPagado": "0,00",
    "saldoFavor": "300.000,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "14/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70892838-7",
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": "146752/201103",
    "saldoPagado": "3.313.414,59",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "10/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70877589-0",
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": "146753/201104",
    "saldoPagado": "0,00",
    "saldoFavor": "200.000,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "14/2/2022",
    "Procesado": "Correcci—n"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-69556267-1",
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": "146754/201105",
    "saldoPagado": "5.635.000,00",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "8/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71245283-4",
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": "146755/201106",
    "saldoPagado": "0,00",
    "saldoFavor": "300.000,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "14/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70840691-7",
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": "146756/201107",
    "saldoPagado": "5.313.414,59",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "15/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9002185787,
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": 3004648676658,
    "saldoPagado": "666.868.000,00",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "14/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "MEX",
    "id_razonsocial": "NPI620328DP8",
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": 220060080649,
    "saldoPagado": "213,35",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA DIGITALES",
    "fecha_procesado": "18/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71509409-2",
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": "146759/201110",
    "saldoPagado": "4.313.414,59",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "16/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013543905,
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": 3004648676658,
    "saldoPagado": "256.200.000,00",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "18/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013161970,
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": 3004648676659,
    "saldoPagado": "3.002.124.000,00",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "11/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9014115336,
    "a–o": 2022,
    "periodo_fiscal": 1,
    "n_verificacion": 3004648676660,
    "saldoPagado": "4.520.000,00",
    "saldoFavor": "",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "14/2/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70892838-7",
    "a–o": 2022,
    "periodo_fiscal": 12,
    "n_verificacion": "146752/201103",
    "saldoPagado": "3.313.414,59",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "10/1/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70877589-0",
    "a–o": 2022,
    "periodo_fiscal": 12,
    "n_verificacion": "146753/201104",
    "saldoPagado": "0,00",
    "saldoFavor": "200.000,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "14/1/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-69556267-1",
    "a–o": 2022,
    "periodo_fiscal": 12,
    "n_verificacion": "146754/201105",
    "saldoPagado": "5.635.000,00",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "8/1/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71245283-4",
    "a–o": 2022,
    "periodo_fiscal": 12,
    "n_verificacion": "146755/201106",
    "saldoPagado": "0,00",
    "saldoFavor": "300.000,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "14/1/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-70840691-7",
    "a–o": 2022,
    "periodo_fiscal": 12,
    "n_verificacion": "146756/201107",
    "saldoPagado": "5.313.414,59",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "15/1/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9002185787,
    "a–o": 2022,
    "periodo_fiscal": 12,
    "n_verificacion": 3004648676658,
    "saldoPagado": "666.868.000,00",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "14/1/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "MEX",
    "id_razonsocial": "NPI620328DP8",
    "a–o": 2022,
    "periodo_fiscal": 12,
    "n_verificacion": 220060080649,
    "saldoPagado": "213,35",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA DIGITALES",
    "fecha_procesado": "18/1/2022",
    "Procesado": "OK Procesado"
  },
  {
    "pais": "ARG",
    "id_razonsocial": "30-71509409-2",
    "a–o": 2022,
    "periodo_fiscal": 12,
    "n_verificacion": "146759/201110",
    "saldoPagado": "4.313.414,59",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "16/1/2022",
    "Procesado": "No procesado"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013543905,
    "a–o": 2022,
    "periodo_fiscal": 12,
    "n_verificacion": 3004648676658,
    "saldoPagado": "256.200.000,00",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "18/1/2022",
    "Procesado": "No procesado"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9013161970,
    "a–o": 2022,
    "periodo_fiscal": 12,
    "n_verificacion": 3004648676659,
    "saldoPagado": "3.002.124.000,00",
    "saldoFavor": "0,00",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "11/1/2022",
    "Procesado": "No procesado"
  },
  {
    "pais": "COL",
    "id_razonsocial": 9014115336,
    "a–o": 2022,
    "periodo_fiscal": 12,
    "n_verificacion": 3004648676660,
    "saldoPagado": "4.520.000,00",
    "saldoFavor": "",
    "Nombre Impuesto": "IVA",
    "fecha_procesado": "14/1/2022",
    "Procesado": "No procesado"
  }
]

In [2]:
from datetime import datetime

contador = 0
grafico1={}  # {2:1,1:1}
for dicc in lista_extraccExcel:#lista diccionarios
    # lista de tipo diccionario
    periodo = dicc['periodo_fiscal'] 
    if periodo in grafico1.keys():#esta en el dicc

        grafico1[periodo] = grafico1[periodo] + 1
    else:
        grafico1[periodo] = 1
print(grafico1,"periodo fiscal" )



grafico1={}  # {2:1,1:1}
for dicc in lista_extraccExcel:#lista diccionarios
    # lista de tipo diccionario
    periodo =  datetime.strptime(dicc['Fecha_vencimiento'], "%d/%m/%Y").month
    if periodo in grafico1.keys():#esta en el dicc

        grafico1[periodo] = grafico1[periodo] + 1
    else:
        grafico1[periodo] = 1
print(grafico1, "Procesamiento de Impuestos por cada mes en curso correcto")



grafico2={} # {"Correccion":3, "No Procesado":4, "Ok Procesado":26}
for dicc1 in lista_procesadopdf:
    procesado = dicc1['Procesado']
    if procesado in grafico2.keys():
        grafico2[procesado] = grafico2[procesado] + 1
    else:
        grafico2[procesado] = 1
print(grafico2, "Estado de documentos en el Procesamiento del Impuesto ")




{2: 14, 1: 19, 12: 11} periodo fiscal
{3: 14, 2: 19, 1: 11} Procesamiento de Impuestos por cada mes en curso correcto
{'OK Procesado': 26, 'Pendiente': 11, 'Correcci—n': 3, 'No procesado': 4} Estado de documentos en el Procesamiento del Impuesto 


In [8]:
print(grafico1)
imagen_uno = {}
for periodo, cantidad in grafico1.items():
    imagen_uno[str(periodo)] = cantidad
print(imagen_uno)

{3: 14, 2: 19, 1: 11, '3': 14}
{'3': 14, '2': 19, '1': 11}


In [13]:
"""Escritura para formar estructura visualización """

def grafica_uno():
    data ={
            'datasets': [{
                'label': 'Procesamiento de Impuestos por mes',
                'data': imagen_uno,
                'backgroundColor': [
                    'rgba(255, 99, 132, 0.2)',
                    'rgba(54, 162, 235, 0.2)',
                    'rgba(255, 206, 86, 0.2)',
                    'rgba(75, 192, 192, 0.2)',
                    'rgba(153, 102, 255, 0.2)',
                    'rgba(255, 159, 64, 0.2)'
                ],
                'borderColor': [
                    'rgba(255, 99, 132, 1)',
                    'rgba(54, 162, 235, 1)',
                    'rgba(255, 206, 86, 1)',
                    'rgba(75, 192, 192, 1)',
                    'rgba(153, 102, 255, 1)',
                    'rgba(255, 159, 64, 1)'
                ],
                'borderWidth': 1
            }]
            }
    return data
print(grafica_uno())


{'datasets': [{'label': 'Procesamiento de Impuestos por mes', 'data': {'3': 14, '2': 19, '1': 11}, 'backgroundColor': ['rgba(255, 99, 132, 0.2)', 'rgba(54, 162, 235, 0.2)', 'rgba(255, 206, 86, 0.2)', 'rgba(75, 192, 192, 0.2)', 'rgba(153, 102, 255, 0.2)', 'rgba(255, 159, 64, 0.2)'], 'borderColor': ['rgba(255, 99, 132, 1)', 'rgba(54, 162, 235, 1)', 'rgba(255, 206, 86, 1)', 'rgba(75, 192, 192, 1)', 'rgba(153, 102, 255, 1)', 'rgba(255, 159, 64, 1)'], 'borderWidth': 1}]}


In [15]:
def grafica_dos():
    data ={
            'datasets': [{
                'label': 'Procesamiento de Impuestos por mes',
                'data': grafico2
            }]
            }
    return data
print(grafica_dos())


{'datasets': [{'label': 'Procesamiento de Impuestos por mes', 'data': {'OK Procesado': 26, 'Pendiente': 11, 'Correcci—n': 3, 'No procesado': 4}}]}


In [16]:
from re import L


lista_estados={}  # {2:1,1:1}
datos = {}
for dicc in lista_extraccExcel:#lista diccionarios
    
    # lista de tipo diccionario
    periodo = dicc['periodo_fiscal']
    id = dicc['id_razonsocial']
    mes =  datetime.strptime(dicc['Fecha_vencimiento'], "%d/%m/%Y").month
    year =  datetime.strptime(dicc['Fecha_vencimiento'], "%d/%m/%Y").year
    if (id,periodo, year) not in datos.keys():
        datos[(id,periodo,year)] = {} #se crea clave para ambas listas
    if mes in datos[(id,periodo,year)].keys():#esta en el dicc
        datos[(id,periodo, year)][mes] = datos[(id,periodo, year)][mes] + 1
    else:
        datos[(id,periodo, year)][mes] = 1
print(datos)






{('30-70892838-7', 2, 2022): {3: 1}, ('30-70877589-0', 2, 2022): {3: 1}, (9013543200, 2, 2022): {3: 1}, (9013161451, 2, 2022): {3: 1}, (9014115458, 2, 2022): {3: 1}, ('30-69556267-1', 2, 2022): {3: 1}, ('30-71245283-4', 2, 2022): {3: 1}, ('30-70840691-7', 2, 2022): {3: 1}, (9002185787, 2, 2022): {3: 1}, ('NPI620328DP8', 2, 2022): {3: 1}, ('30-71509409-2', 2, 2022): {3: 1}, (9013543905, 2, 2022): {3: 1}, (9013161970, 2, 2022): {3: 1}, (9014115336, 2, 2022): {3: 1}, ('30-70892838-7', 1, 2022): {2: 3}, ('30-70877589-0', 1, 2022): {2: 3}, ('30-69556267-1', 1, 2022): {2: 3}, ('30-71245283-4', 1, 2022): {2: 3}, ('30-70840691-7', 1, 2022): {2: 1}, (9002185787, 1, 2022): {2: 1}, ('NPI620328DP8', 1, 2022): {2: 1}, ('30-71509409-2', 1, 2022): {2: 1}, (9013543905, 1, 2022): {2: 1}, (9013161970, 1, 2022): {2: 1}, (9014115336, 1, 2022): {2: 1}, ('30-70892838-7', 12, 2022): {1: 1}, ('30-70877589-0', 12, 2022): {1: 1}, ('30-69556267-1', 12, 2022): {1: 1}, ('30-71245283-4', 12, 2022): {1: 1}, ('30-708

In [17]:
datos_pdf = {}
for dicc_pdf in lista_procesadopdf: 
    periodo = dicc_pdf['periodo_fiscal']
    id = dicc_pdf['id_razonsocial']
    año = dicc_pdf['a–o'] #ojo toca ver como leer "ñ"
    estado = dicc_pdf['Procesado']
    if estado not in datos_pdf:
        datos_pdf[estado] = [(id,periodo,year)] #se crea clave para ambas listas
    elif (id,periodo,year) not in datos_pdf[estado]:
        datos_pdf[estado].append((id,periodo,year))
print(datos_pdf)

{'OK Procesado': [('30-70892838-7', 2, 2022), (9002185787, 2, 2022), ('NPI620328DP8', 2, 2022), ('30-70892838-7', 1, 2022), ('30-69556267-1', 1, 2022), ('30-71245283-4', 1, 2022), ('30-70840691-7', 1, 2022), (9002185787, 1, 2022), ('NPI620328DP8', 1, 2022), ('30-71509409-2', 1, 2022), (9013543905, 1, 2022), (9013161970, 1, 2022), (9014115336, 1, 2022), ('30-70892838-7', 12, 2022), ('30-70877589-0', 12, 2022), ('30-69556267-1', 12, 2022), ('30-71245283-4', 12, 2022), ('30-70840691-7', 12, 2022), (9002185787, 12, 2022), ('NPI620328DP8', 12, 2022)], 'Pendiente': [('30-70877589-0', 2, 2022), (9013543200, 2, 2022), (9013161451, 2, 2022), (9014115458, 2, 2022), ('30-69556267-1', 2, 2022), ('30-71245283-4', 2, 2022), ('30-70840691-7', 2, 2022), ('30-71509409-2', 2, 2022), (9013543905, 2, 2022), (9013161970, 2, 2022), (9014115336, 2, 2022)], 'Correcci—n': [('30-70877589-0', 1, 2022)], 'No procesado': [('30-71509409-2', 12, 2022), (9013543905, 12, 2022), (9013161970, 12, 2022), (9014115336, 12,

In [18]:
# procesamiento de grafico
datos_grafico3 = []
for c, v in datos_pdf.items():
    #print(c)
    data = {'label': c} # dicionarior por estdo de proceso.
    data['data'] = {} # diccionario que contiene los meses.
    for clave_excel in v:
        if clave_excel in datos.keys():
            
            mes, cantidad = list(zip(datos[clave_excel].keys(), datos[clave_excel].values()))[0]
            # print(datos[clave_excel], mes, cantidad)
            if mes not in data['data'].keys():
                data['data'][mes] = cantidad
            elif mes in data['data'].keys():
                data['data'][mes] += cantidad
    datos_grafico3.append(data)

print(datos_grafico3)
    


[{'label': 'OK Procesado', 'data': {3: 3, 2: 16, 1: 7}}, {'label': 'Pendiente', 'data': {3: 11}}, {'label': 'Correcci—n', 'data': {2: 3}}, {'label': 'No procesado', 'data': {1: 4}}]


In [75]:
lista_grafica_tres =[]

z=0
for dicc in datos_grafico3: # recorro lista datasets
    i = 0
    dataset = {} 
    for c, v  in dicc.items(): # recorre el dataset
        # 1 c='label', v='Ok procesado' 
        # 2 c='data' , v='{}'
        
        if c == 'label':
            dataset[c] = v
        if c == 'data': # partición
            dataset['data'] = v
            imagen_tres={}
            for periodo, cantidad in dicc[c].items():
                imagen_tres[str(periodo)] = cantidad
            dataset['data'] = imagen_tres
        print(f'dataset {z}', f'campos {i}', c,v, imagen_tres)
        i += 1
    lista_grafica_tres.append(dataset)
    z += 1

#print(lista_grafica_tres)
# print(lista_grafica_tres[1])
# print(lista_grafica_tres[2])
#print(dataset)





dataset 0 campos 0 label OK Procesado {'1': 4}
dataset 0 campos 1 data {3: 3, 2: 16, 1: 7} {'3': 3, '2': 16, '1': 7}
dataset 1 campos 0 label Pendiente {'3': 3, '2': 16, '1': 7}
dataset 1 campos 1 data {3: 11} {'3': 11}
dataset 2 campos 0 label Correcci—n {'3': 11}
dataset 2 campos 1 data {2: 3} {'2': 3}
dataset 3 campos 0 label No procesado {'2': 3}
dataset 3 campos 1 data {1: 4} {'1': 4}


In [ ]:
lista_grafica_tres =[]


for dicc in datos_grafico3: # recorro lista datasets

    dataset = {} 
    for c, v  in dicc.items(): # recorre el dataset

        if c == 'label':
            dataset[c] = v
        if c == 'data': # partición
            dataset['data'] = v
            imagen_tres={}
            for periodo, cantidad in dicc[c].items():
                imagen_tres[str(periodo)] = cantidad
            dataset['data'] = imagen_tres
       
    lista_grafica_tres.append(dataset)
print(lista_grafica_tres)
# print(lista_grafica_tres[1])
# print(lista_grafica_tres[2])
#print(dataset)

In [86]:
#grafico 4 parte uno separar nombre revisor
# No funcionó



# lista_estados={}  # {2:1,1:1}
# datos = {}
# for dicc in lista_extraccExcel:#lista diccionarios
    
#     # lista de tipo diccionario
#     nombre_revisor = dicc['Revisor']
#     id = dicc['id_razonsocial']
#     periodo = dicc['periodo_fiscal']
#     year =  datetime.strptime(dicc['Fecha_vencimiento'], "%d/%m/%Y").year

#     if (id,periodo, year) not in datos.keys():
#         datos[(id,periodo,year)] = {} #se crea clave para ambas listas
#     if nombre_revisor in datos[(id,periodo,year)].keys():#esta en el dicc
#         datos[(id,periodo, year)][nombre_revisor] = datos[(id,periodo, year)][nombre_revisor] + 1
#     else:
#         datos[(id,periodo, year)][nombre_revisor] = 1

# print(datos)



In [87]:
datos_revisor ={} ## Se utilizó para el grafico de revisores

for fila in lista_extraccExcel:
    nombre_revisor = fila['Revisor']
    id = fila['id_razonsocial']
    periodo = fila['periodo_fiscal']
    year =  datetime.strptime(fila['Fecha_vencimiento'], "%d/%m/%Y").year

    if  nombre_revisor not in  datos_revisor.keys():
        datos_revisor[nombre_revisor] = [(id,periodo, year)]
    elif (id,periodo, year) not in datos_revisor[nombre_revisor]:
        datos_revisor[nombre_revisor].append((id,periodo, year))
print(datos_revisor)


{'Juan Espinosa': [('30-70892838-7', 2, 2022), (9013543200, 2, 2022), ('30-70840691-7', 2, 2022), (9013543905, 2, 2022), ('30-70892838-7', 1, 2022), ('30-70840691-7', 1, 2022), (9013543905, 1, 2022), ('30-70892838-7', 12, 2022), ('30-70840691-7', 12, 2022), (9013543905, 12, 2022)], 'Camilo Sarmiento': [('30-70877589-0', 2, 2022), (9013161451, 2, 2022), (9002185787, 2, 2022), (9013161970, 2, 2022), ('30-70877589-0', 1, 2022), (9002185787, 1, 2022), (9013161970, 1, 2022), ('30-70877589-0', 12, 2022), (9002185787, 12, 2022), (9013161970, 12, 2022)], 'Flor Cardenas': [(9014115458, 2, 2022), ('30-69556267-1', 2, 2022), ('NPI620328DP8', 2, 2022), (9014115336, 2, 2022), ('30-69556267-1', 1, 2022), ('NPI620328DP8', 1, 2022), (9014115336, 1, 2022), ('30-69556267-1', 12, 2022), ('NPI620328DP8', 12, 2022), (9014115336, 12, 2022)], 'Maria Rivera': [('30-71245283-4', 2, 2022), ('30-71509409-2', 2, 2022), ('30-71245283-4', 1, 2022), ('30-71509409-2', 1, 2022), ('30-71245283-4', 12, 2022), ('30-71509

In [88]:
# datos_revisor_periodo ={}

# for fila in lista_extraccExcel:
#     nombre_revisor = fila['Revisor']
#     id = fila['id_razonsocial']
#     periodo = fila['periodo_fiscal']
#     year =  datetime.strptime(fila['Fecha_vencimiento'], "%d/%m/%Y").year
#     if  (id,periodo, year) not in  datos_revisor_periodo.keys():
#         datos_revisor_periodo[(id,periodo, year)] = nombre_revisor
#     # elif (id,periodo, year) not in datos_revisor_periodo[nombre_revisor]:
#     #     datos_revisor_periodo[nombre_revisor].append((id,periodo, year))
# print(datos_revisor_periodo)

In [89]:

grafico_revisor = []


for clave_revisor, v in datos_revisor.items():
    data = {'label': clave_revisor} # dicionarior por estdo de proceso.
    data['data'] = {} # diccionario que contiene los meses.
    for clave_unica in v:
        for clave_estado, valores_clave in datos_pdf.items():
            if clave_unica in valores_clave:
                if clave_estado not in data['data'].keys():
                    data['data'][clave_estado] = 1
                else:
                    data['data'][clave_estado] = data['data'][clave_estado] + 1
    grafico_revisor.append(data)


print(grafico_revisor)

[{'label': 'Juan Espinosa', 'data': {'OK Procesado': 6, 'Pendiente': 3, 'No procesado': 1}}, {'label': 'Camilo Sarmiento', 'data': {'Pendiente': 3, 'OK Procesado': 5, 'Correcci—n': 1, 'No procesado': 1}}, {'label': 'Flor Cardenas', 'data': {'Pendiente': 3, 'OK Procesado': 6, 'No procesado': 1}}, {'label': 'Maria Rivera', 'data': {'Pendiente': 2, 'OK Procesado': 3, 'No procesado': 1}}]
